# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [2]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [3]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure! Here's a joke for you: Why was the math book sad? Because it had too many problems.


In [4]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Sure, which joke would you like me to repeat?


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [5]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [6]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Great to meet you, Joe! Let's dive into some humor. You know Joe, I'm constantly trying to improve myself. Just last week I enrolled in a self-defense class. Turns out, my biggest enemy is gluten.

--------------------------------------------------------------------------------
joe (to cathy):

Gluten, always sneaking up on you when you least expect it! That's like me trying to improve myself by taking up yoga, only to realize that my biggest enemy is downward dog.

--------------------------------------------------------------------------------
cathy (to joe):

Haha, downward dog strikes again! But hey, at least you're getting some exercise in, even if it's just fighting off a yoga pose. I've been trying to eat healthier lately, but healthy eating can be so confusing. I mean, is it still a salad if I add extra cheese and baco

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [7]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': "Great to meet you, Joe! Let's dive into some humor. You know "
             "Joe, I'm constantly trying to improve myself. Just last week I "
             'enrolled in a self-defense class. Turns out, my biggest enemy is '
             'gluten.',
  'role': 'user'},
 {'content': 'Gluten, always sneaking up on you when you least expect it! '
             "That's like me trying to improve myself by taking up yoga, only "
             'to realize that my biggest enemy is downward dog.',
  'role': 'assistant'},
 {'content': "Haha, downward dog strikes again! But hey, at least you're "
             "getting some exercise in, even if it's just fighting off a yoga "
             "pose. I've been trying to eat healthier lately, but healthy "
             'eating can be so confusing. I mean, is it still a salad if I add '
             'extra cheese and bacon?',
  'role': 'user'}]


In [8]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 149,
                                                             'cost': 0.0003555,
                                                             'prompt_tokens': 264,
                                                             'total_tokens': 413},
                                      'total_cost': 0.0003555},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 149,
                                                             'cost': 0.0003555,
                                                             'prompt_tokens': 264,
                                                             'total_tokens': 413},
                                      'total_cost': 0.0003555}}


In [9]:
pprint.pprint(chat_result.summary)

("Haha, downward dog strikes again! But hey, at least you're getting some "
 "exercise in, even if it's just fighting off a yoga pose. I've been trying to "
 'eat healthier lately, but healthy eating can be so confusing. I mean, is it '
 'still a salad if I add extra cheese and bacon?')


## Get a better summary of the conversation

In [10]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Great to meet you, Joe! Let's dive into some humor. You know Joe, I'm constantly trying to improve myself. Just last week I enrolled in a self-defense class. Turns out, my biggest enemy is gluten.

--------------------------------------------------------------------------------
joe (to cathy):

Gluten, always sneaking up on you when you least expect it! That's like me trying to improve myself by taking up yoga, only to realize that my biggest enemy is downward dog.

--------------------------------------------------------------------------------
cathy (to joe):

Haha, downward dog strikes again! But hey, at least you're getting some exercise in, even if it's just fighting off a yoga pose. I've been trying to eat healthier lately, but healthy eating can be so confusing. I mean, is it still a salad if I add extra cheese and baco

In [11]:
pprint.pprint(chat_result.summary)

('Joe and Cathy share humor in jokes about self-improvement, with Joe joking '
 'about gluten being his enemy and Cathy quipping about yoga poses. They also '
 'discuss the challenges of eating healthy, questioning whether a salad with '
 'extra cheese and bacon still counts as healthy.')


## Chat Termination

Chat can be terminated using a termination conditions.

In [12]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [13]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, I'm always ready to make you laugh! What do you get when you cross a snowman and a vampire? Frostbite!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! How do you come up with these jokes?

--------------------------------------------------------------------------------
cathy (to joe):

I gotta say, Joe, my brain is just wired for finding humor in everything! Life is just one big comedy show, and I'm always looking for the punchline!

--------------------------------------------------------------------------------
joe (to cathy):

That's a great outlook to have, Cathy! Seeing the humor in life is so important. Keep spreading those laughs!

--------------------------------------------------------------------------------
cathy (to joe):

T

In [ ]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
